In [1]:
import numpy as np
import pandas as pd


In [2]:
index_df = pd.read_csv("cache/index_balanced.csv")
index_df

,path,label,fold_id,source_filename,clip_id,window_id,pipeline_name,augment_desc,source_type
0,/workspace/cache/mel64/glass_breaking/fold1/1-...,glass_breaking,1,1-20133-A-39.wav,1-20133-A-39,w00,base,base,glass_base
1,/workspace/cache/mel64/glass_breaking/fold1/1-...,glass_breaking,1,1-20133-A-39.wav,1-20133-A-39,w00,shift_gain,shift_gain(time_shift+gain),glass_aug
2,/workspace/cache/mel64/glass_breaking/fold1/1-...,glass_breaking,1,1-20133-A-39.wav,1-20133-A-39,w00,shift_gain,shift_gain(time_shift+gain),glass_aug
3,/workspace/cache/mel64/glass_breaking/fold1/1-...,glass_breaking,1,1-20133-A-39.wav,1-20133-A-39,w00,stretch_reverb,stretch_reverb(time_stretch+reverb),glass_aug
4,/workspace/cache/mel64/glass_breaking/fold1/1-...,glass_breaking,1,1-20133-A-39.wav,1-20133-A-39,w00,stretch_reverb,stretch_reverb(time_stretch+reverb),glass_aug
...,...,...,...,...,...,...,...,...,...
1161,/workspace/cache/mel64/background/fold5/5-2614...,background,5,5-261464-A-23.wav,5-261464-A-23,w00,base,base,background_raw
1162,/workspace/cache/mel64/background/fold5/5-2356...,background,5,5-235671-A-38.wav,5-235671-A-38,w00,base,base,background_raw
1163,/workspace/cache/mel64/background/fold5/5-1820...,background,5,5-182010-A-36.wav,5-182010-A-36,w07,base,base,background_raw
1164,/workspace/cache/mel64/background/fold5/5-2389...,background,5,5-238938-A-27.wav,5-238938-A-27,w02,base,base,background_raw
